# Setup

## Imports

In [1]:
from vaiutils import path_consts

All the Keras functionality

In [2]:
from keras.layers import Input, Embedding, Dense, Flatten, SimpleRNN, TimeDistributed
from keras.layers.merge import add
from keras.initializers import identity
from keras.models import Model, Sequential
from keras.backend import clear_session
from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


## Load data and useful variables

In [3]:
for k, v in path_consts('TextGen', 'TextGen'):
    exec(k + '=v')

In [134]:
data = open(DIR_DATA + '/pokemon.txt').read()

chars = sorted(list(set(data)))
chars.insert(0, '\0')

vocab_size = len(chars)

print("Vocabulary size:", vocab_size, "\nCharacters:", ''.join(chars))
print("\nCorpus length:", len(data), "\nSample:\n", data[:1000])

Vocabulary size: 63 
Characters:  
 '-.2:ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzé♀♂

Corpus length: 6791 
Sample:
 Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran♀
Nidorina
Nidoqueen
Nidoran♂
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetch'd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubon

In [135]:
char_idx = {c: i for i, c in enumerate(chars)}

In [136]:
data = [char_idx[c] for c in data]

# 3 Character Model

## Create inputs

Create a list of every fourth character

In [137]:
cs = 3
x1 = np.stack([data[i] for i in range(0, len(data) - cs - 1, cs)])
x2 = np.stack([data[i + 1] for i in range(0, len(data) - cs - 1, cs)])
x3 = np.stack([data[i + 2] for i in range(0, len(data) - cs - 1, cs)])
y = np.stack([data[i + 3] for i in range(0, len(data) - cs - 1, cs)])

## Create Model

Embedding function that creates inputs and embeddings

In [138]:
def create_embedding(name, n_in, n_out):
    inp = Input(name=name + '_in', dtype='int64', shape=[1])
    return inp, Flatten()(Embedding(n_in, n_out, input_length=1, name=name)(inp))

In [139]:
clear_session()

n_factors = 32
c1_in, c1 = create_embedding('c1', vocab_size, n_factors)
c2_in, c2 = create_embedding('c2', vocab_size, n_factors)
c3_in, c3 = create_embedding('c3', vocab_size, n_factors)

n_hidden = 256

dense_in = Dense(n_hidden, activation='relu')
c1_hidden = dense_in(c1)

dense_hidden = Dense(n_hidden, activation='tanh')

c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = add([c2_dense, hidden_2])

c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = add([c3_dense, hidden_3])

dense_out = Dense(vocab_size, activation='softmax')
c4_out = dense_out(c3_hidden)

In [140]:
model = Model([c1_in, c2_in, c3_in], c4_out)
model.compile('adam', loss='sparse_categorical_crossentropy')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c3_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c2_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c1_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c3 (Embedding)                   (None, 1, 32)         2016        c3_in[0][0]                      
___________________________________________________________________________________________

## Train Model

Helper function to fit the model

In [141]:
def fit_model(epochs=1, lr=1e-3):
    model.optimizer.lr = lr
    model.fit([x1, x2, x3], y, batch_size=64, epochs=epochs,
              verbose=0, callbacks=[TQDMNotebookCallback()], shuffle=False)

In [142]:
fit_model(8)

## Test Model

Helper function to get fourth predicted character given three

In [143]:
def get_next_char(inp):
    idxs = [char_idx[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.random.choice(range(vocab_size), p=p.ravel())
    return chars[i]

In [181]:
sample_string = 'ivy'
for _ in range(10):
    sample_string = sample_string + get_next_char(sample_string[-3:])
print(sample_string)

ivynf
Aante
B


# Simple Character Model

## Create Inputs

In [89]:
cs = 8
x = [np.stack([data[i + n] for i in range(0, len(data) - cs - 1, cs)])
     for n in range(cs)]
y = np.stack([data[i + cs] for i in range(0, len(data) - cs - 1, cs)])

## Create Model

In [90]:
clear_session()
c_ins = [create_embedding('c' + str(n), vocab_size, n_factors)
         for n in range(cs)]

dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu',
                     kernel_initializer=identity())
dense_out = Dense(vocab_size, activation='softmax')

hidden = dense_in(c_ins[0][1])

for i in range(1, cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = add([c_dense, hidden])

c_out = dense_out(hidden)

In [91]:
model = Model([c[0] for c in c_ins], c_out)
model.compile('adam', loss='sparse_categorical_crossentropy')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c7_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c6_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c5_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c4_in (InputLayer)               (None, 1)             0                                            
___________________________________________________________________________________________

## Train Model

Model fitting utility function

In [92]:
def fit_model(epochs=1, lr=1e-3):
    model.optimizer.lr = lr
    model.fit(x, y, batch_size=64, epochs=epochs,
              verbose=0, callbacks=[TQDMNotebookCallback()])

In [93]:
fit_model(8)

## Test Model

In [112]:
sample_start_idx = randint(0, len(data) - cs)
sample_string = ''.join([chars[i]
                         for i in data[sample_start_idx:sample_start_idx + cs]])
for _ in range(20):
    sample_string = sample_string + get_next_char(sample_string[-cs:])
print(sample_string)

u
abhinani
rakha
vineer
gani


# Keras RNN

Create Sequential Model

In [113]:
clear_session()
model = Sequential([Embedding(vocab_size, n_factors, input_length=cs),
                    SimpleRNN(n_hidden, activation='relu',
                              recurrent_initializer=identity()),
                    Dense(vocab_size, activation='softmax')])
model.compile('adam', loss='sparse_categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8, 32)             3136      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               73984     
_________________________________________________________________
dense_1 (Dense)              (None, 98)                25186     
Total params: 102,306
Trainable params: 102,306
Non-trainable params: 0
_________________________________________________________________


Train Model

In [114]:
def fit_model(epochs=1, lr=1e-3):
    model.optimizer.lr = lr
    model.fit(np.concatenate(x, 1), y, batch_size=64, epochs=epochs,
              verbose=0, callbacks=[TQDMNotebookCallback()])

In [115]:
fit_model(8)

Test Model

In [116]:
def get_next_char(inp):
    idxs = [char_idx[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(np.concatenate(arrs)[np.newaxis])
    i = np.random.choice(range(vocab_size), p=p.ravel())
    return chars[i]

In [120]:
sample_start_idx = randint(0, len(data) - cs)
sample_string = ''.join([chars[i]
                         for i in data[sample_start_idx:sample_start_idx + cs]])
for _ in range(100):
    sample_string = sample_string + get_next_char(sample_string[-cs:])
print(sample_string)

a
varisha sharal,abhi
sojala
laseeba
ajpiy @ niona
priyankisa
saidiy chagu doin sipna
dipni
rekha devi
savin


# Seq-2-Seq Model

Create Inputs

In [121]:
cs = 8
x = [np.stack([data[i + n] for i in range(0, len(data) - cs - 1, cs)])
     for n in range(cs)]
y = [np.stack([data[i + n] for i in range(1, len(data) - cs, cs)])
     for n in range(cs)]

Create Model

In [122]:
clear_session()
model = Sequential([Embedding(vocab_size, n_factors, input_length=cs),
                    SimpleRNN(n_hidden, activation='relu', return_sequences=True,
                              recurrent_initializer=identity()),
                    TimeDistributed(Dense(vocab_size, activation='softmax'))])
model.compile('adam', loss='sparse_categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8, 32)             3136      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 8, 256)            73984     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 98)             25186     
Total params: 102,306
Trainable params: 102,306
Non-trainable params: 0
_________________________________________________________________


Train Model

In [123]:
def fit_model(epochs=1, lr=1e-3):
    model.optimizer.lr = lr
    model.fit(np.stack(x, 1), np.expand_dims(np.stack(y, 1), 2), batch_size=64, epochs=epochs,
              verbose=0, callbacks=[TQDMNotebookCallback()])

In [124]:
fit_model(8)

In [125]:
def get_next_char(inp):
    idxs = [char_idx[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(np.concatenate(arrs)[np.newaxis])[0]
    idxs = [np.random.choice(range(vocab_size), p=prob.ravel()) for prob in p]
    return ''.join(chars[i] for i in idxs)

In [133]:
sample_start_idx = randint(0, len(data) - cs)
sample_string = ''.join([chars[i]
                         for i in data[sample_start_idx:sample_start_idx + cs]])
for _ in range(10):
    sample_string = sample_string + get_next_char(sample_string[-cs:])
print(sample_string)

li
smt. 
thht varopaesanfh

shmttaashe... 

a d  jgbraibmhoha

 ratannks j
v

 ikarirmss


# Stateful Model

Create Model

In [ ]:
clear_session()
model = Sequential([Embedding(vocab_size, n_factors, input_length=cs, batch_input_shape=(64, cs)),
                    SimpleRNN(n_hidden, activation='relu', recurrent_initializer=identity(
                    ), return_sequences=True),
                    TimeDistributed(Dense(vocab_size, activation='softmax'))])
model.compile('adam', loss='sparse_categorical_crossentropy')
model.summary()

Train Model

In [ ]:
max_len = x[0].shape[0] - (x[0].shape[0] % 64)

In [ ]:
def fit_model(epochs=1, lr=1e-3):
    model.optimizer.lr = lr
    model.fit(np.stack(x, 1)[:max_len], np.expand_dims(np.stack(y, 1), 2)[:max_len], batch_size=64, epochs=epochs,
              verbose=0, callbacks=[TQDMNotebookCallback()], shuffle=False)

In [ ]:
fit_model(8)